In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [2]:
import datetime as dt
import psychropy

# YEAR = 'TMY'
START_MON = '1'
START_DAY = '1'
END_MON = '12'
END_DAY = '31'

# Input and output file location
# location = 'LA'
LOC_WEATHER_FILE = 'USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw'

CSV_FILE = os.path.join('future_weather', 'LA_HEATWAVES', 'LA_HW_Midterm_MostSevere_2051.csv')
EPW_FILE = os.path.join('future_weather', 'LA_HEATWAVES', 'LA_HW_Midterm_MostSevere_2051.epw')
csv_headers_epw_map = {6: 'tas', 7:'dew', 8:'hurs', 9:'ps', 
                       13: 'global_hori', 14: 'dir_norm', 15: 'dif_hori', 20:'wind_dir', 21:'sfcWind'}
days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [3]:
df_csv_result = pd.read_csv(CSV_FILE, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [4]:
df_csv_result.head()

,tas,ps,hurs,sfcWind,rsds,wind_dir,dir_norm,dif_hori,dir_hori
time_lst,,,,,,,,,
2051-01-01 00:00:00,8.519762,100580.5058,100.000000,4.000106,0.059571,218,0.0,0.059571,0.0
2051-01-01 01:00:00,9.343634,100533.6172,100.000000,3.100304,0.057487,207,0.0,0.057487,0.0
2051-01-01 02:00:00,9.258816,100582.3291,100.000000,2.238655,0.092862,204,0.0,0.092862,0.0
2051-01-01 03:00:00,9.287439,100583.2111,95.082580,1.785613,0.050764,265,0.0,0.050764,0.0
2051-01-01 04:00:00,10.286558,100583.1368,92.628751,1.444499,0.068279,218,0.0,0.068279,0.0


In [5]:
df_csv_result.keys()

Index(['tas', 'ps', 'hurs', 'sfcWind', 'rsds', 'wind_dir', 'dir_norm',
       'dif_hori', 'dir_hori'],
      dtype='object')

In [6]:
# write EPW
data_to_write = dict()
for i, row in df_csv_result.iterrows():
    row['hum_rat'] = psychropy.Hum_rat2(float(row['tas']), float(row['hurs']), float(row['ps']))
    row['dew'] = psychropy.Dew_point(float(row['ps']), float(row['hum_rat']))
    row['global_hori'] = row['dif_hori'] + row['dir_hori']
    mon = i.month
    day = i.day
    hour = i.hour
    year = i.year
    row['year'] = year
#     print(str(mon) + "/" + str(day) + "/" + year + ' ' +  str(hour) + ':00:00')
    data_to_write[str(mon) + "/" + str(day) + "/" + str(hour)] = row

In [7]:
data_to_write['1/1/0']

tas                 8.519762
ps             100580.505800
hurs              100.000000
sfcWind             4.000106
rsds                0.059571
wind_dir          218.000000
dir_norm            0.000000
dif_hori            0.059571
dir_hori            0.000000
hum_rat             0.000688
dew               102.446306
global_hori         0.059571
year             2051.000000
Name: 2051-01-01 00:00:00, dtype: float64

In [8]:
import math
f = open(EPW_FILE, 'w')

with open(LOC_WEATHER_FILE, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in csvreader:
        if (len(row) == 35):
            mon = row[1]
            day = row[2]
            hour = row[3]
            minute = row[4]
            # parse 8/1/18 0:00 to 7/31/18 24:00
            if int(hour) == 24:
                hour = 0
                if int(day) == days_in_mon[int(mon)]:
                    mon = int(mon) + 1
                    day = 1
                else:
                    day = int(day) + 1

            # Notes: The format of the CSV time stamp
            cur_time_stamp = str(mon) + "/" + str(day) + "/" + str(hour)
            if cur_time_stamp in data_to_write:
                cur_data_to_write = data_to_write[cur_time_stamp]
                row[0] = str(cur_data_to_write['year'])

                for epw_col in csv_headers_epw_map:
                    if csv_headers_epw_map[epw_col] in cur_data_to_write:
                        try:
                            row[epw_col] = str(round(cur_data_to_write[csv_headers_epw_map[epw_col]], 3))
                        except:
                            row[epw_col] = str(cur_data_to_write[csv_headers_epw_map[epw_col]])
            else:
                print(cur_time_stamp + ' not found.')
        f.write(','.join(row) + '\n')

f.close()

13/1/0 not found.
